## Connect to API

In [25]:
import configparser
import pandas as pd
from datetime import datetime 
import requests
from xml.etree import cElementTree as ET

# Setting config values
config = configparser.ConfigParser()
config.read("/Users/dankoban/Documents/Dissertation/config.ini")
api_key = config['awis']['secret_key']
access_id = config['awis']['account_id']

API Helper Functions

In [26]:
class XmlDictConfig(dict):
    '''
    Example usage:

    >>> tree = ElementTree.parse('your_file.xml')
    >>> root = tree.getroot()
    >>> xmldict = XmlDictConfig(root)

    Or, if you want to use an XML string:

    >>> root = ElementTree.XML(xml_string)
    >>> xmldict = XmlDictConfig(root)

    And then use xmldict for what it is... a dict.
    '''
    def __init__(self, parent_element):
        if parent_element.items():
            self.update(dict(parent_element.items()))
        for element in parent_element:
            if element:
                # treat like dict - we assume that if the first two tags
                # in a series are different, then they are all different.
                if len(element) == 1 or element[0].tag != element[1].tag:
                    aDict = XmlDictConfig(element)
                # treat like list - we assume that if the first two tags
                # in a series are the same, then the rest are the same.
                else:
                    # here, we put the list in dictionary; the key is the
                    # tag name the list elements all share in common, and
                    # the value is the list itself 
                    aDict = {element[0].tag: XmlListConfig(element)}
                # if the tag has attributes, add those to the dict
                if element.items():
                    aDict.update(dict(element.items()))
                self.update({element.tag: aDict})
            # this assumes that if you've got an attribute in a tag,
            # you won't be having any text. This may or may not be a 
            # good idea -- time will tell. It works for the way we are
            # currently doing XML configuration files...
            elif element.items():
                self.update({element.tag: dict(element.items())})
            # finally, if there are no child tags and no attributes, extract
            # the text
            else:
                self.update({element.tag: element.text})
                
class XmlListConfig(list):
    def __init__(self, aList):
        for element in aList:
            if element:
                # treat like dict
                if len(element) == 1 or element[0].tag != element[1].tag:
                    self.append(XmlDictConfig(element))
                # treat like list
                elif element[0].tag == element[1].tag:
                    self.append(XmlListConfig(element))
            elif element.text:
                text = element.text.strip()
                if text:
                    self.append(text)
                    
def get_alexa_rank(url, api_key, access_id):
    request_url = 'https://awis.api.alexa.com/api?Action=urlInfo&ResponseGroup=Rank&Url=' + url
    headers = {'Accept':'application/xml',
               'Content-Type': 'application/xml',
               'X-Amz-Date': datetime.utcnow().strftime('%Y%m%dT%H%M%SZ'),
               'Authorization': 'AWS4-HMAC-SHA256 Credential=' + access_id + '/' + 
                                datetime.utcnow().strftime('%Y%m%d') + 
                                '/us-east-1/execute-api/aws4_request,SignedHeaders=host;x-amz-date',
               'x-api-key': api_key
              }
    r = requests.get(request_url, headers=headers)
    root = ET.XML(r.text)
    dic = XmlDictConfig(root)
    dic = dic['Results']['Result']['Alexa']['TrafficData']
    return dic

## Query domains for Alexa Rank

In [28]:
domains = ['cnn.com', 'dailywire.com', 'publichealth.lacounty.gov', 'hannity.com']
for domain in domains:
    print(get_alexa_rank(domain, api_key = awis_token, access_id = awis_access_id))

{'DataUrl': 'cnn.com/', 'Rank': '110'}
{'DataUrl': 'dailywire.com/', 'Rank': '3142'}
{'DataUrl': 'publichealth.lacounty.gov/', 'Rank': '8926'}
{'DataUrl': 'hannity.com/', 'Rank': '65783'}
